Preprocessing of historical production data: discard data of unwanted power plants, retain monthly files - old version, because too many nan values or missing rows detected

In [ ]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Basisverzeichnisse
input_dir = r"E:\MA_data\raw production history ENTSO-E"
output_dir = r"C:\Users\alexa\Documents\Webapp\data\production_history\processed_old"

# Funktion zum Auffüllen fehlender Stunden und Zählen der fehlenden Werte
def fill_missing_hours(data, start_time, end_time):
    # Erstelle eine vollständige Zeitreihe für den Monat
    full_time_range = pd.date_range(start=start_time, end=end_time, freq='h')

    # Umwandeln der 'DateTime (UTC)'-Spalte in datetime-Objekte
    data.loc[:, 'DateTime (UTC)'] = pd.to_datetime(data['DateTime (UTC)'])

    # Setze den Index auf die DateTime-Spalte
    data.set_index('DateTime (UTC)', inplace=True)
    
    # Reindexiere die Daten, um fehlende Stunden mit NaN aufzufüllen
    data = data.reindex(full_time_range)

    # Zähle die Anzahl der fehlenden Werte
    missing_count = data['ActualGenerationOutput(MW)'].isna().sum()
    
    # Fülle fehlende Werte mit 0 (sowohl gerade hinzugefügte Zeilen mit nan Werten, als auch bereits bestehende Zeilen, in denen schon vorher keine Werte für die Produktion waren)
    data['ActualGenerationOutput(MW)'] = data['ActualGenerationOutput(MW)'].fillna(0)
    
    # Setze den Index zurück
    data.reset_index(inplace=True)
    
    return data, missing_count

number_missing_values = []

# Liste der Monate von 2015-01 bis 2024-10 generieren
months = pd.date_range(start="2015-01", end="2015-02", freq="MS").strftime("%Y_%m").tolist()

# For-Schleife für jede Datei
for i, month in enumerate(months):

    # Dateipfad erstellen
    input_file = os.path.join(input_dir, f"{month}_ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1.csv")
    output_file = os.path.join(output_dir, f"production_summary_{month}.xlsx")

    # Datei einlesen
    print(f"Bearbeite Datei: {input_file}")
    data = pd.read_csv(input_file, sep='\t')

    # Filtere nach GenerationUnitType == 'Wind Onshore' oder 'Wind Offshore'
    filtered_data = data[(data['GenerationUnitType'] == 'Wind Onshore ') | (data['GenerationUnitType'] == 'Wind Offshore ')]

    # Wichtige Spalten identifizieren, 'AreaCode', 'AreaDisplayName', 'AreaTypeCode' and 'MapCode' of identical WPPs may differ --> use at least one of them as a criterion to identify unique windfarms, and sort out the duplicates manually, because otherwise, the production data are appended twice to the same wind farm
    unique_windfarms = filtered_data[['GenerationUnitName', 'GenerationUnitCode', 'GenerationUnitType', 'GenerationUnitInstalledCapacity(MW)', 'AreaCode']].drop_duplicates()
    
    # Auffüllen fehlender Stunden und Zählen der fehlenden Werte
    start_time = data['DateTime (UTC)'].min()
    end_time = data['DateTime (UTC)'].max()

    number_missing_values.append([])

    # Listen für die Produktion zu jeder Stunde hinzufügen
    production_data = []
    for _, row in unique_windfarms.iterrows():
        windfarm_data = filtered_data[(filtered_data['GenerationUnitName'] == row['GenerationUnitName']) & (filtered_data['AreaCode'] == row['AreaCode'])]
        windfarm_data, missing_count = fill_missing_hours(windfarm_data, start_time, end_time)
        production_values = windfarm_data['ActualGenerationOutput(MW)'].tolist() # where production is nan, the WPP has consumed and not generator power
        row_data = {
            'GenerationUnitName': row['GenerationUnitName'],
            'GenerationUnitCode': row['GenerationUnitCode'],
            'GenerationUnitType': row['GenerationUnitType'],
            'GenerationUnitInstalledCapacity(MW)': row['GenerationUnitInstalledCapacity(MW)'],
            'Production (MW)': production_values
        }
        production_data.append(row_data)

        # Zähle die fehlenden Werte für diesen Monat
        number_missing_values[i].append(missing_count)

    # DataFrame erstellen und in Excel speichern
    output_df = pd.DataFrame(production_data)
    output_df.to_excel(output_file, index=False)

    print("Anzahl der fehlenden Werte je Windkraftwerk für diesen Monat:", number_missing_values[i])
    print(f"Excel-Datei wurde erfolgreich erstellt: {output_file}")

Bearbeite Datei: E:\MA_data\raw production history ENTSO-E\2015_02_ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1.csv
Excel-Datei wurde erfolgreich erstellt: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_02.xlsx
Bearbeite Datei: E:\MA_data\raw production history ENTSO-E\2015_03_ActualGenerationOutputPerGenerationUnit_16.1.A_r2.1.csv
Excel-Datei wurde erfolgreich erstellt: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_03.xlsx


save the number_missing_values as excel

In [ ]:
# Mache die Liste flach, um alle Elemente in einer einzigen Liste zu sammeln
number_missing_values_flat = [item for sublist in number_missing_values for item in sublist]

# Berechne den Durchschnitt
average = sum(number_missing_values_flat) / len(number_missing_values_flat)
print(f"Der Durchschnitt ist: {average}")

# Überschrift
column_name = "number of missing elements per wind power plant for all investigated months"

# DataFrame erstellen
df = pd.DataFrame({column_name: [str(sublist) for sublist in number_missing_values]})

# Datei speichern
output_file = r"data\number_missing_values.xlsx"  # Pfad und Dateiname anpassen
df.to_excel(output_file, index=False)

print(f"Die Excel-Datei wurde erfolgreich gespeichert: {output_file}")

merge all monthly production data files to one combined file

In [ ]:
import pandas as pd
import os

# Verzeichnisse
input_dir = r"C:\Users\alexa\Documents\Webapp\data\production_history\processed"
output_file = r"C:\Users\alexa\Documents\Webapp\data\production_history\production_summary_all.xlsx"

# Liste der Monate von 2015_01 bis 2024_10
months = pd.date_range(start="2015-01", end="2024-10", freq="MS").strftime("%Y_%m").tolist()

columns_old = ['GenerationUnitName', 'GenerationUnitCode', 'GenerationUnitType', 'GenerationUnitInstalledCapacity(MW)', 'AreaDisplayName', 'MapCode', 'AreaTypeCode']
# Leeres DataFrame für das Endergebnis
columns = columns_old + months
final_df = pd.DataFrame(columns=columns)

# Einlesen der einzelnen Dateien
for month in months:
    input_file = os.path.join(input_dir, f"production_summary_{month}.xlsx")

    # Überprüfen, ob die Datei existiert
    if not os.path.exists(input_file):
        print(f"Datei nicht gefunden: {input_file}")
        continue

    # Datei einlesen
    print(f"Verarbeite Datei: {input_file}")
    df = pd.read_excel(input_file)

    # Sicherstellen, dass die Spalte 'Production (MW)' existiert
    if 'Production (MW)' not in df.columns:
        print(f"Spalte 'Production (MW)' fehlt in {input_file}")
        continue

    df.rename(columns={'Production (MW)': month}, inplace=True)

    # Zusammenführen der Daten
    if final_df.empty:
        final_df = df
    else:
        # Zusammenführen: Gleiche Windkraftanlagen zusammenführen, neue hinzufügen
        final_df = pd.merge(final_df, df, how='outer', on=columns_old)

# Excel-Tabelle speichern
final_df.to_excel(output_file, index=False)
print(f"Zusammengeführte Excel-Tabelle wurde erfolgreich gespeichert unter: {output_file}")

Verarbeite Datei: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_01.xlsx
Verarbeite Datei: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_02.xlsx
Verarbeite Datei: C:\Users\alexa\Documents\Webapp\data\production_history\processed_new\production_summary_2015_03.xlsx
Zusammengeführte Excel-Tabelle wurde gespeichert unter: C:\Users\alexa\Documents\Webapp\data\production_history\production_summary_all.xlsx


Visualise filling rate of production data file (1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Datei laden
file_path = r"C:\Users\alexa\Documents\Webapp\data\production_history\processed\production_summary_all.xlsx"
df = pd.read_excel(file_path)

# Anzahl der Windkraftanlagen pro AreaDisplayName
counts = df['AreaDisplayName'].value_counts()

# Berechnung der prozentualen Ausfüllquote pro AreaDisplayName
percentages = {}
for area in counts.index:
    subset = df[df['AreaDisplayName'] == area]
    total_cells = len(subset) * (len(subset.columns) - 6)  # Exkludiere nicht-produktive Spalten
    filled_cells = subset.iloc[:, 6:].notna().sum().sum()  # Nur Produktionsdaten berücksichtigen
    percentages[area] = (filled_cells / total_cells) * 100

# Balkendiagramm erstellen
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(counts.index, counts.values, color='skyblue')
ax.set_title('Anzahl der Windkraftanlagen mit Produktionsdaten pro Land')
ax.set_xlabel('Land (AreaDisplayName)')
ax.set_ylabel('Anzahl der Windkraftanlagen')
ax.tick_params(axis='x', rotation=45)

# Prozentsätze als Text hinzufügen
for bar, area in zip(bars, counts.index):
    height = bar.get_height()
    percentage = percentages[area]
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{percentage:.1f} %', ha='center', va='bottom')

plt.tight_layout()
plt.show()

Visualise filling rate of production data file (2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Datei laden
file_path = r"C:\Users\alexa\Documents\Webapp\data\production_history\processed\production_summary_all.xlsx"
df = pd.read_excel(file_path)

# Liste der Monatskolumnen
month_columns = [col for col in df.columns if col.startswith("20")]

# Anzahl der Windkraftanlagen pro AreaDisplayName (Land)
windfarm_count = df.groupby("AreaDisplayName").size()

# Durchschnittliche Ausfüllquote pro Land
fill_rates = df[month_columns].notna().mean(axis=1)  # Berechne pro Windkraftanlage
average_fill_rate_per_country = df.groupby("AreaDisplayName")[month_columns].apply(
    lambda x: x.notna().mean(axis=1).mean()
)

# Plot erstellen
fig, ax1 = plt.subplots(figsize=(12, 6))

# Linke y-Achse: Anzahl der Windkraftanlagen
ax1.bar(
    windfarm_count.index,
    windfarm_count.values,
    label="Anzahl der Windkraftanlagen",
    alpha=0.7
)
ax1.set_ylabel("Anzahl der Windkraftanlagen", fontsize=12)
ax1.set_xlabel("AreaDisplayName (Land)", fontsize=12)
ax1.tick_params(axis="x", rotation=45)
ax1.legend(loc="upper left")

# Rechte y-Achse: Durchschnittliche Ausfüllquote
ax2 = ax1.twinx()
ax2.bar(
    average_fill_rate_per_country.index,
    average_fill_rate_per_country.values * 100,  # Prozentualer Wert
    label="Durchschnittliche Ausfüllquote (%)",
    alpha=0.5,
    color="orange"
)
ax2.set_ylabel("Durchschnittliche Ausfüllquote (%)", fontsize=12)
ax2.legend(loc="upper right")

# Titel und Layout
plt.title("Windkraftanlagen und Ausfüllquote pro Land", fontsize=14)
plt.tight_layout()

# Plot anzeigen
plt.show()

(Steps with assigning a meaningful name to UK WPPs just as in current Preprocessing file. Then perform manual assignment to The Wind Power database.)

assign ENTSO-E WPPs to The Wind Power WPPs

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

df_wind_power = pd.read_excel("data/WPPs+production.xlsx")
df_assignment = pd.read_excel("data/Assignment_old.xlsx")

In [ ]:
# Filtere nur Zeilen, bei denen "ID_The-Wind-Power" nicht "not found" ist
df_assignment = df_assignment[df_assignment["ID_The-Wind-Power"] != "not found"]

# Extrahiere und entpacke alle gültigen IDs aus der Spalte "ID_The-Wind-Power"
def extract_ids(value):
    # Überprüfen, ob der Wert eine Liste ist, und ggf. in einzelne IDs zerlegen
    if isinstance(value, str) and value.startswith("[") and value.endswith("]"):
        return eval(value)  # Konvertiert die Zeichenkette in eine Liste
    elif isinstance(value, (int, str)):
        return [int(value)]  # Einzelne IDs werden in eine Liste gewandelt
    return []

valid_ids = set()
df_assignment["ID_The-Wind-Power"].apply(lambda x: valid_ids.update(extract_ids(x)))

df_filtered = df_wind_power[df_wind_power['ID'].isin(valid_ids)].copy()
actual_ids = set(df_filtered['ID'])
suspended_ids = valid_ids - actual_ids

print("number potential WPPs:", len(valid_ids))
print("number actual WPPs:", len(actual_ids))
print("number suspended WPPs (no name, location, capacity or status not in operation):", len(suspended_ids))

# Füge Spalten für Produktion von 2015_01 bis 2024_10 hinzu
new_columns = {f"{year}_{month:02d}": [[] for _ in range(len(df_filtered))]
            for year in range(2015, 2025) for month in range(1, 13)
            if f"{year}_{month:02d}" in df_assignment.columns}

# Füge die neuen Spalten zum DataFrame hinzu
df_filtered = pd.concat([df_filtered, pd.DataFrame(new_columns, index=df_filtered.index)], axis=1)

# Gehe durch jede Zeile der Assignment-Datei und füge Produktionsdaten hinzu
for _, row in df_assignment.iterrows():
    
    ids_in_row = extract_ids(row["ID_The-Wind-Power"])
    first_id = ids_in_row[0]

    if first_id in suspended_ids:
        continue # jump to next iteration, because following line would fail for suspended_ids

    current_index = df_filtered.loc[df_filtered['ID'] == first_id].index[0]

    # add production values for each month, only requires the first ID
    for year in range(2015, 2025):
        for month in range(1, 13):
            column_name = f"{year}_{month:02d}"
            if column_name in df_assignment.columns: # neglect 2024_11 and 2024_12
                production_month = row[column_name]
                if isinstance(production_month, str): # type(value) = str, meaning value = production values
                    production_month = production_month.replace("nan", "0") # where production is nan, the WPP has consumed and not generator power
                    production_month = eval(production_month) # [[]] <-- "[[]]"
                    production_month = production_month[0] # [] <-- [[]]
                    existing_production = df_filtered.at[current_index, column_name]
                    
                    if existing_production == []: # no production values in cell for this month
                        df_filtered.at[current_index, column_name] = production_month
                    else: # several production values to be added to one WPP
                        combined_production = [a + b for a, b in zip(existing_production, production_month)]
                        df_filtered.at[current_index, column_name] = combined_production

    # add capacities of WPPs, if several are assigned to one row in the assignment table
    if first_id in actual_ids and len(ids_in_row) > 1: # only treat every id once here, because rows are discarded
        total_power = 0
        for id in ids_in_row:
            total_power += df_filtered.loc[df_filtered['ID'] == id, "Total power"].item() # add power
            if id != first_id:
                df_filtered = df_filtered[df_filtered['ID'] != id] # delete from dataframe
        df_filtered.loc[df_filtered['ID'] == first_id, "Total power"] = total_power
    
    # keep track of treated IDs to not try to delete rows twice 
    for id in ids_in_row:
        if id in actual_ids:
            actual_ids.discard(id)

actual_cluster_ids = set(df_filtered['ID'])
print("number WPPs after clustering", len(actual_cluster_ids))
df_filtered.to_excel("data/WPPs+production.xlsx", index=False)